In [7]:
import os
import glob
import numpy as np
import pandas as pd
import gc
import calendar
import datetime

import cudf
import cupy
import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.schema import Schema, Tags
import nvtabular as nv

In [8]:
from nvtabular.ops import *


In [4]:
month1 = "202202"
month3 = "202204"
mydata = cudf.read_parquet(f"s3a://zarklab-token-recommender/buy-sell-updated-{month1}-{month3}")


In [20]:
# Initialize Dataset with s3 parquet data
# dataset = nv.Dataset("s3:///zarklab-hive-test/buy-sell-updated/pt_d=2023*/*",engine="parquet")
mydata = cudf.read_parquet("s3a://zarklab-hive-test/buy-sell-updated/pt_d=2023*/*.parquet")


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/ec2-user/SageMaker/.persisted_conda/transformers4rec-23.04/include/rmm/mr/device/cuda_memory_resource.hpp

In [12]:
data2 = nv.Dataset('/zarklab-hive-test/buy-sell-updated/pt_d=2023*/*',engine = 'parquet')


IndexError: list index out of range

# remove duplicate

In [4]:
mydata['timestamp'] = cudf.to_datetime(mydata['timestamp'])


In [5]:
print("Count with  repeated interactions: {}".format(len(mydata)))

# Sorts the dataframe by session and timestamp, to remove consecutive repetitions
# mydata.timestamp = mydata.timestamp.astype(int)
interactions_df = mydata.sort_values(['recipient', 'timestamp'])
past_ids = interactions_df['buyAsset'].shift(1).fillna()
session_past_ids = interactions_df['recipient'].shift(1).fillna()

# Keeping only no consecutive repeated in session interactions
interactions_df = interactions_df[~((interactions_df['recipient'] == session_past_ids) & (interactions_df['buyAsset'] == past_ids))]

print("Count after removed in-session repeated interactions: {}".format(len(interactions_df)))

Count with  repeated interactions: 1725084
Count after removed in-session repeated interactions: 955723


In [6]:
interactions_df.head()

,timestamp,recipient,txFee_eth,buyAsset,buyQty1,buyPrice,merge,combined_token_name,combined_token_symbol,token_category,token_rank_category,risky_flags,bluecheckmark
1664529,2022-03-12 06:41:35,0x000000000000000000000000000000000000005f,0.004351,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.07,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
409331,2022-03-17 03:49:58,0x00000000000000000000000000000000000000e0,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.62,17.64,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
1302535,2022-03-13 10:02:30,0x00000000000000000000000000000000000000f1,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.03,0.88,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
963579,2022-03-10 23:21:45,0x0000000000000000000000000000000000000139,0.004353,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.12,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
765950,2022-03-18 00:27:11,0x000000000000000000000000000000000000016e,0.004354,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,3.38,94.19,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE


In [7]:
dataset = nv.Dataset(interactions_df)

In [8]:
dataset.head()

,timestamp,recipient,txFee_eth,buyAsset,buyQty1,buyPrice,merge,combined_token_name,combined_token_symbol,token_category,token_rank_category,risky_flags,bluecheckmark
1664529,2022-03-12 06:41:35,0x000000000000000000000000000000000000005f,0.004351,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.07,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
409331,2022-03-17 03:49:58,0x00000000000000000000000000000000000000e0,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.62,17.64,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
1302535,2022-03-13 10:02:30,0x00000000000000000000000000000000000000f1,0.004352,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.03,0.88,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
963579,2022-03-10 23:21:45,0x0000000000000000000000000000000000000139,0.004353,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,0.00,0.12,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE
765950,2022-03-18 00:27:11,0x000000000000000000000000000000000000016e,0.004354,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,3.38,94.19,0x1776e1f26f98b1a5df9cd347953a26dd3cb46671,Numeraire,nmr,Crowdfunding and Launchpads,Nano,0,TRUE


In [17]:
SESSIONS_MAX_LENGTH = 1

# Encodes categorical features as contiguous integers
cat_feats = ColumnSelector(['buyAsset']) >> nvt.ops.Categorify()



In [18]:
# create time features
session_ts = ColumnSelector(['timestamp'])
session_time = (
    session_ts >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col)) >> 
    nvt.ops.Rename(name = 'timestamp')
)


In [19]:

buyPrice_norm =  ColumnSelector(['buyPrice']) >> nvt.ops.Rename(name='buyPrice_log_norm')


In [20]:
features = ColumnSelector(['recipient']) + buyPrice_norm +session_time+cat_feats


In [21]:
# Define Groupby Operator
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["recipient"], 
    sort_cols=["timestamp"],
    aggs={
        'buyAsset': ["list", "count"],
        'timestamp': ["first"],  
        'buyPrice_log_norm': ["list"]
    },
    name_sep="-")

In [22]:
# Select and truncate the sequential features


sequence_features_truncated_item = (
    groupby_features['buyAsset-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['buyPrice_log_norm-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)


In [23]:
MINIMUM_SESSION_LENGTH = 1
selected_features = (
    groupby_features['recipient','buyAsset-count', 'timestamp-first' ] + 
    sequence_features_truncated_item +
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["buyAsset-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['buyAsset-list', 'buyPrice_log_norm-list'] >>  nvt.ops.ValueCount()

workflow = nvt.Workflow(filtered_sessions['recipient', 'timestamp-first'] + seq_feats_list)

In [24]:
#workflow = nvt.Workflow.load('./preprocessWorkflow')
workflow.fit(dataset)


In [25]:
out = workflow.transform(dataset)

In [27]:
out.head(60)

,recipient,timestamp-first,buyAsset-list,buyPrice_log_norm-list
0,0x000000000000000000000000000000000000005f,2022-03-12 06:41:35,[478],[0.07]
1,0x00000000000000000000000000000000000000e0,2022-03-17 03:49:58,[478],[17.64]
2,0x00000000000000000000000000000000000000f1,2022-03-13 10:02:30,[478],[0.88]
3,0x0000000000000000000000000000000000000139,2022-03-10 23:21:45,[478],[0.12]
4,0x000000000000000000000000000000000000016e,2022-03-18 00:27:11,[478],[94.19]
5,0x0000000000000000000000000000000000000185,2022-03-12 21:26:02,[478],[3.49]
6,0x00000000000000000000000000000000000001f0,2022-03-11 03:07:06,[478],[0.14]
7,0x00000000000000000000000000000000000002e7,2022-03-14 15:53:11,[478],[0.42]
8,0x0000000000000000000000000000000000000336,2022-03-10 23:30:33,[478],[51.52]
9,0x0000000000000000000000000000000000000340,2022-03-18 00:29:22,[478],[19.14]


In [87]:
# workflow.save('./preprocessWorkflow')

In [28]:
out.to_parquet(f'./data/price/data.parquet')